<a href="https://colab.research.google.com/github/hjori66/Kaist-AI605-2021-Spring/blob/main/KAIST_AI605_Assignment_2_20194364.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KAIST AI605 Assignment 2: Token Classification with RNNs and Attention
Author: Minjoon Seo (minjoon@kaist.ac.kr)

TA in charge: Taehyung Kwon (taehyung.kwon@kaist.ac.kr)

**Due date**:  April 19 (Mon) 11:00pm, 2021  


Your name: Taehwan Kim

Your student ID: 20194364

Your collaborators: -

## Assignment Objectives
- Verify theoretically and empirically how Transformer's attention mechanism works for sequence modeling task.
- Implement Transformer's encoder attention layer from scratch using PyTorch.
- Design an Attention-based token classification model using PyTorch.
- Apply the token classification model to a popular machine reading comprehension task, Stanford Question Answering Dataset (SQuAD).
- (Bonus) Analyze pros and cons between using RNN + attention versus purely attention.

## Your Submission
Your submission will be a link to a Colab notebook that has all written answers and is fully executable. You will submit your assignment via KLMS. Use in-line LaTeX (see below) for mathematical expressions. Collaboration among students is allowed but it is not a group assignment so make sure your answer and code are your own. Also make sure to mention your collaborators in your assignment with their names and their student ids.

## Grading
The entire assignment is out of 100 points. There are two bonus questions with 30 points altogether. Your final score can be higher than 100 points.


## Environment
You will only use Python 3.7 and PyTorch 1.8, which is already available on Colab:

In [1]:
from platform import python_version
import torch

print("python", python_version())
print("torch", torch.__version__)

python 3.7.10
torch 1.8.1+cu101


## 1. Transformer's Attention Layer

We will first start with going over a few concepts that you learned in your high school statistics class. The variance of a random variable $X$, $\text{Var}(X)$ is defined as $\text{E}[(X-\mu)^2]$ where $\mu$ is the mean of $X$. Furthermore, given two independent random variables $X$ and $Y$ and a constant $a$,
$$ \text{Var}(X+Y) = \text{Var}(X) + \text{Var}(Y), \quad \ldots \; \text{(1)}$$ 
$$ \text{Var}(aX) = a^2\text{Var}(X), \quad \ldots \; \text{(2)}$$
$$ \text{Var}(XY) = \text{E}(X^2)\text{E}(Y^2) - [\text{E}(X)]^2[\text{E}(Y)]^2. \quad \ldots \; \text{(3)}$$

**Problem 1.1** *(10 points)* Suppose we are given two sets of $n$ random variables, $X_1 \dots X_n$ and $Y_1 \dots Y_n$, where all of these $2n$ variables are mutually independent and have a mean of $0$ and a variance of $1$. Prove that
$$\text{Var}\left(\sum_i^n X_i Y_i\right) = n.$$

There is a typo in the formula $\text{(2)}$. I changed it.

**Answer 1.1** 

$$
\begin{align}
  \text{Var}\left(\sum_i^n X_i Y_i\right)
  &= \sum_i^n \text{Var} \left(X_i Y_i\right) \quad \because \text{(1), independence} \\
  &= \sum_i^n \left[\text{E}(X_i^2)\text{E}(Y_i^2) - [\text{E}(X_i)]^2[\text{E}(Y_i)]^2 \right] \quad \because \text{(3)} \\
  &= \sum_i^n \left[\text{E}((X_i-0)^2)\text{E}((Y_i-0)^2)\right] \\
  &= \sum_i^n \left[\text{E}((X_i-[\text{E}(X_i)])^2)\text{E}((Y_i-[\text{E}(Y_i)])^2)\right] \\
  &= \sum_i^n \left[\text{Var}(X_i) \text{Var}(Y_i)\right] \\
  &= \sum_i^n \left[1\right] = n \\
\end{align}
\\
$$

In Lecture 08 and 09, we discussed how the attention is computed in Transformer via the following equation,
$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V.$$
**Problem 1.2** *(10 points)*  Suppose $Q$ and $K$ are matrices of independent variables each of which has a mean of $0$ and a variance of $1$. Using what you learned from Problem 1.1., show that
$$\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) = 1.$$

**Answer 1.2** 

$$
\begin{align}
  \text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right)
  &= \left(\frac{1}{\sqrt{d_k}}\right)^2 \text{Var}\left(QK^\top\right)  \quad \because \text{(2)} \\
  &= \frac{1}{d_k} \text{Var}\left(QK^\top\right) \\
\end{align}
\\
\\
$$

Then, we can focus on the one element of $QK$, $\left(QK\right)_{ij}$ without loss of generality.

$$
\begin{align}
  \frac{1}{d_k} \text{Var}\left(\left(QK\right)_{ij}^\top\right)
  &= \frac{1}{d_k} \text{Var}\left(\sum_{t}^{d_k} \left(Q_{it} K_{tj}\right) \right) \\
  &= \frac{1}{d_k} \left(\sum_{t}^{d_k} \text{Var}\left(Q_{it} K_{tj}\right) \right) \quad \because \text{(1), } Q_{it} \text{ and } Y_{tj} \text{ are mutually independent} \\
  &= \frac{1}{d_k} \left(d_k\right) = 1 \quad \because \text{Problem 1.1} \\
\end{align}
\\
$$

Therefore, 

$$
\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) = 1.
$$



**Problem 1.3** *(10 points)* What would happen if the assumption that the variance of $Q$ and $K$ is $1$ does not hold? Consider each case of it being higher and lower than $1$ and conjecture what it implies, respectively.

**Answer 1.3** \

If the variance of $Q_{ij}$ and $K_{ij}$ is higher than 1 for all i and j, then

$$
\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) > 1.
$$

Then, the variance of the output of the decoder becomes larger.
If we use the softmax function on this output, then the final result might be "too" sharp. (Actually, this is not true, because of the residual connection) \
So, I guess that the model overfits faster than original model.

\

Otherwse, if the variance of $Q_{ij}$ and $K_{ij}$ is lower than 1 for all i and j, then

$$
\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) < 1.
$$

Then, the variance of the output of the decoder becomes smaller.
If we use the softmax function on this output, then the final result might be "too" smooth. 
\
So, I guess that the early training would be more unstable than normal although we use the bigger learning rate. The training time would be longer than the original version. 

## 2. Preprocessing SQuAD

We will use `datasets` package offered by Hugging Face, which allows us to easily download various language datasets, including Stanford Question Answering Dataset (SQuAD).

First, install the package:

In [2]:
!pip install datasets

     |████████████████████████████████| 204kB 7.8MB/s 
     |████████████████████████████████| 112kB 14.9MB/s 
     |████████████████████████████████| 245kB 11.1MB/s 


Then, download SQuAD and print the first example:

In [3]:
from datasets import load_dataset
squad_dataset = load_dataset('squad')
print(squad_dataset['train'][0])

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4fffa6cf76083860f85fa83486ec3028e7e32c342c218ff2a620fc6b2868483a. Subsequent calls will reuse this data.
{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}, 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'id': '57

Here, `answer_start` corresponds to the character-level start position of the answer, and `text` is the answer text itself. You will note that `answer_start` and `text` fields are given as lists but they only contain one item each. In fact, you can safely assume that this is the case for the training data. During evaluation, however, you will utilize several possible answers so that your evaluation can be compared against all of them. So your code need to handle multiple-answers case as well.

As we discussed in Lecture 05, we want to formulate this task as a token classification problem. That is, we want to find which token of the context corresponds to the start position of the answer, and which corresponds to the end.

**Problem 2.1** *(10 points)* Write `preprocess()` function that takes a SQuAD example as the input and outputs space-tokenized context and question, as well as the start and end token position of the answer if it has the answer field. That is, a pseudo code would look like:
```python
def preprocess(example):
  out = {'context': ['each', 'token'], 
         'question': ['each', 'token']}
  if 'answers' not in example:
    return out
  out['answers'] = [{'start': 3, 'end': 5}]
  return out
```
Verify that this code works by comparing between the original answer text and the concatenation of the answer tokens from start to end in training data. Report the percentage of the questions that have exact match.

**Answer 2.1**



In [4]:
def preprocess(example):
    def tokenizer(sentence):
        if sentence is None:
            return list()
        return sentence.split()

    out = dict()
    context = example['context']
    question = example['question']

    out['context'] = tokenizer(context)
    out['question'] = tokenizer(question)

    answer_list = list()
    for answer_index in range(len(example['answers']['text'])):
        answer = example['answers']['text'][answer_index]
        answer_start = example['answers']['answer_start'][answer_index]
        answer_end = answer_start + len(answer)

        if (answer_start == 0 or context[answer_start-1] == ' ') \
            and (answer_end == len(context) or context[answer_end] == ' '):
            n_tokens_before_answer = len(tokenizer(context[:answer_start]))
            n_tokens_answer = len(tokenizer(answer))
            answer_list.append({'start': n_tokens_before_answer, 'end': n_tokens_before_answer+n_tokens_answer-1})
    
    if not answer_list:
        out['answers'] = answer_list

    return out

n_exact_match = 0.0
for i, example in enumerate(squad_dataset['train']):
    out = preprocess(example)
    if 'answers' in out.keys():
        n_exact_match += 1

print("number of answers that have exact match : ", n_exact_match)
print("number of training data : ", len(squad_dataset['train']))
print("the percentage of the exact match (training) : ", n_exact_match / len(squad_dataset['train']))

# number of answers that have exact match :  41616.0
# number of training data :  87599
# the percentage of the exact match :  0.47507391636890833 -> pretty low


number of answers that have exact match :  41616.0
number of training data :  87599
the percentage of the exact match (training) :  0.47507391636890833


We want to maximize the percentage of the exact match. You might see a low percentage however, due to bad tokenization. For instance, such space-based tokenization will fail to separate between "world" and "!" in "hello world!". 

**Problem 2.2** *(10 points)* Write an advanced tokenization model that always separates non-alphabet characters as independent tokens. For instance, "hello1 world!!" will be tokenized into "hello", "1", "world", "!", and "!". Using this new tokenizer, re-run the `preprocess` function and report the exact match percentage. How does the ratio change?

**Answer 2.2**



In [5]:
def find_nonalpha_list(dataset):
    nonalpha_list = list()
    for example in dataset:
        for c in example['context']:
            if not c.isalpha() and c not in nonalpha_list:
                nonalpha_list.append(c)
        for c in example['question']:
            if not c.isalpha() and c not in nonalpha_list:
                nonalpha_list.append(c)
    return nonalpha_list


def preprocess(example, nonalpha_list):
    def tokenizer(sentence, nonalpha_list):
        if sentence is None:
            return list()

        for nonalpha_token in nonalpha_list:
            sentence = sentence.replace(nonalpha_token, ' ' + nonalpha_token + ' ')

        sentence = ' '.join(sentence.split())
        return sentence.split()
        
    out = dict()
    context = example['context']
    question = example['question']
    id = example['id']

    out['context'] = tokenizer(context, nonalpha_list)
    out['question'] = tokenizer(question, nonalpha_list)
    out['id'] = id

    answer_list = list()
    for answer_index in range(len(example['answers']['text'])):
        answer = example['answers']['text'][answer_index]
        answer_start = example['answers']['answer_start'][answer_index]
        answer_end = answer_start + len(answer)

        if (answer_start == 0 or context[answer_start-1] in nonalpha_list) \
            and (answer_end == len(context) or context[answer_end] in nonalpha_list):
            n_tokens_before_answer = len(tokenizer(context[:answer_start], nonalpha_list))
            n_tokens_answer = len(tokenizer(answer, nonalpha_list))
            answer_list.append({'start': n_tokens_before_answer, 'end': n_tokens_before_answer+n_tokens_answer-1})

    if answer_list:
        out['answers'] = answer_list

    return out

dataset = squad_dataset['train']
# dataset = squad_dataset['validation'] # Do this if you want to check the valid dataset

nonalpha_list = find_nonalpha_list(squad_dataset['train'])
print("nonalpha_list : ", nonalpha_list)

n_exact_match = 0.0
for i, example in enumerate(dataset):
  out = preprocess(example, nonalpha_list)
  if 'answers' in out.keys():
    n_exact_match += 1

print("number of answers that have exact match : ", n_exact_match)
print("number of training data : ", len(dataset))
print("the percentage of the exact match (training) : ", n_exact_match / len(dataset))


# nonalpha_list :  [',', ' ', '.', "'", '"', '1', '8', '5', '(', '3', ')', '?', '-', '7', '6', '9', '2', '0', ';', '–', '&', '4', '%', '$', '[', ']', '/', ':', '#', '—', '!', '“', '’', '”', '<', '\u200b', '̃', '£', '½', '+', '¢', '−', '°', '>', '€', '《', '》', '±', '~', '¥', '²', '❤', '=', '\u200e', '͡', '́', '`', '्', 'ु', 'ः', 'ॊ', 'ि', 'ा', '\u200d', '\u200c', '*', '‘', '\u3000', '•', '§', '⁄', '\n', '̯', '̩', '…', '·', 'ָ', 'ִ', 'ׁ', 'ַ', 'ּ', 'ְ', 'ّ', '⟨', '◌', '⟩', '˭', '̤', '♠', '∅', '̞', '×', '̥', '′', '″', '\ufeff', '_', 'ֿ', '´', '^', '̧', '̄', '→', '‑', '，', '₹', '\u202f', '♯', '₂', '₥', '⁊', '\u2009', '{', '}', '|', '@', '̪', '‚', '›', 'ׂ', 'ֵ', 'ِ', 'ْ', 'َ', '̍', '˥', '˨', '˩', '¡', '√', '¿', 'ာ', 'း', 'ُ', '≥', '˚', '≈', '⋅', 'ี', '︘', '�', '～', '〜', '̀', 'ོ', '་', '˧', 'ಾ', 'ು', '್', 'া', '্', 'ಿ', '∗', '∈', '≡', '∖', '№', '÷', 'ٔ', '¶', 'ิ', '₤', '♆', '⅓', '∝', '¼', 'ٍ', 'ֹ', '̌', '。', '̠', '₯']
# number of answers that have exact match :  87108.0
# number of training data :  87599
# the percentage of the exact match (training) :  0.9943949131839405 -> now, it is OK

# number of answers that have exact match :  10566.0
# number of validation data :  10570
# the percentage of the exact match (validation) :  0.9996215704824977 -> Also, it is OK


nonalpha_list :  [',', ' ', '.', "'", '"', '1', '8', '5', '(', '3', ')', '?', '-', '7', '6', '9', '2', '0', ';', '–', '&', '4', '%', '$', '[', ']', '/', ':', '#', '—', '!', '“', '’', '”', '<', '\u200b', '̃', '£', '½', '+', '¢', '−', '°', '>', '€', '《', '》', '±', '~', '¥', '²', '❤', '=', '\u200e', '͡', '́', '`', '्', 'ु', 'ः', 'ॊ', 'ि', 'ा', '\u200d', '\u200c', '*', '‘', '\u3000', '•', '§', '⁄', '\n', '̯', '̩', '…', '·', 'ָ', 'ִ', 'ׁ', 'ַ', 'ּ', 'ְ', 'ّ', '⟨', '◌', '⟩', '˭', '̤', '♠', '∅', '̞', '×', '̥', '′', '″', '\ufeff', '_', 'ֿ', '´', '^', '̧', '̄', '→', '‑', '，', '₹', '\u202f', '♯', '₂', '₥', '⁊', '\u2009', '{', '}', '|', '@', '̪', '‚', '›', 'ׂ', 'ֵ', 'ِ', 'ْ', 'َ', '̍', '˥', '˨', '˩', '¡', '√', '¿', 'ာ', 'း', 'ُ', '≥', '˚', '≈', '⋅', 'ี', '︘', '�', '～', '〜', '̀', 'ོ', '་', '˧', 'ಾ', 'ು', '್', 'া', '্', 'ಿ', '∗', '∈', '≡', '∖', '№', '÷', 'ٔ', '¶', 'ิ', '₤', '♆', '⅓', '∝', '¼', 'ٍ', 'ֹ', '̌', '。', '̠', '₯']
number of answers that have exact match :  87108.0
number of training data :

## 3. LSTM Baseline for SQuAD

We will bring and reuse our model from Assignment 1. There are two key differences, however. First, we need to classify each token instead of the entire sentence. Second, we have two inputs (context and question) instead of just one.  

In [6]:
# My model from Assignment 1 is too slow to use it.
# I will use torchtext and torch.nn.lstm in the assignment 2.

import torchtext
from torchtext.legacy import data
from torchtext.legacy import datasets
from torchtext.legacy.data import BucketIterator


class SQuAD1Dataset(data.Dataset):
  """
  Defines a dataset for squad1.0.
  """
  
  @staticmethod
  def sort_key(ex):
    return data.interleave_keys(len(ex.context), len(ex.question))

  def __init__(self, data_list, fields, use_bos=True, max_length=None, **kwargs):
    if not isinstance(fields[0], (tuple, list)):
      fields = [('context', fields[0]), 
                ('question', fields[1]), 
                # ('context_question', fields[2]), # For Problem 3.2+, put the question after the context
                ('answer_start', fields[2]), 
                ('answer_end', fields[3]), 
                ('id_index', fields[4])
                ]

    examples = []
    nonalpha_list = find_nonalpha_list(data_list)

    self.id_list = list()
    self.reference = list()

    for _, example in enumerate(data_list):
        out = preprocess(example, nonalpha_list)
        # use data if the answer exists
        if max_length and max_length < max(len(out['context']), len(out['question'])):
            continue
        if 'answers' in out.keys():
            answer_start = out['answers'][0]['start'] # Use index 0 for instant valid accuracy
            answer_end = out['answers'][0]['end'] # Use index 0 for instant valid accuracy
            if use_bos:
                answer_start += 1 # for <BOS> token
                answer_end += 1 # for <BOS> token
            examples.append(data.Example.fromlist([out['context'], 
                                                   out['question'], 
                                                #    out['context'] + ["<CLS>"] + out['question'], # Use <CLS> token
                                                   answer_start,
                                                   answer_end,
                                                   len(self.id_list)], 
                                                  fields))
            self.id_list.append(out['id'])
            self.reference.append({'id':example['id'], 'answers':example['answers']})

    super(SQuAD1Dataset, self).__init__(examples, fields, **kwargs)


class SQuAD1Dataloader():
  """
  Make the dataloader for SQuAD 1.0
  """
  def __init__(self, train_data=None, valid_data=None, batch_size=64, device='cpu', 
                max_length=255, min_freq=2, fix_length=None,
                use_bos=True, use_eos=True, shuffle=True
              ):

    super(SQuAD1Dataloader, self).__init__()

    self.text = data.Field(sequential=True, use_vocab=True, batch_first=True, 
                           include_lengths=True, fix_length=fix_length, 
                           init_token='<BOS>' if use_bos else None, 
                           eos_token='<EOS>' if use_eos else None
                          )
    self.answer_start = data.Field(sequential = False, use_vocab = False)
    self.answer_end = data.Field(sequential = False, use_vocab = False)
    self.id_index = data.Field(sequential = False, use_vocab = False)
    
    train = SQuAD1Dataset(data_list=train_data, 
                          fields = [('context', self.text),
                                    ('question', self.text),
                                    # ('context_question', self.text),
                                    ('answer_start', self.answer_start),
                                    ('answer_end', self.answer_end),
                                    ('id_index', self.id_index)
                                    ], 
                          max_length = max_length
                          )
    valid = SQuAD1Dataset(data_list=valid_data, 
                          fields = [('context', self.text),
                                    ('question', self.text),
                                    # ('context_question', self.text),
                                    ('answer_start', self.answer_start),
                                    ('answer_end', self.answer_end),
                                    ('id_index', self.id_index)
                                    ], 
                          max_length = max_length
                          )
    self.train_id_list = train.id_list
    self.valid_id_list = valid.id_list

    self.train_reference = train.reference
    self.valid_reference = valid.reference
    
    self.train_iter = data.BucketIterator(train, batch_size=batch_size,
                                          device=device,
                                          shuffle=shuffle,
                                          sort_key=lambda x: len(x.question) + (max_length * len(x.context)), 
                                          sort_within_batch = True
                                          )
    self.valid_iter = data.BucketIterator(valid, batch_size=batch_size,
                                          device=device,
                                          shuffle=shuffle,
                                          sort_key=lambda x: len(x.question) + (max_length * len(x.context)), 
                                          sort_within_batch = True
                                          )
    
    self.text.build_vocab(train)


train_dataset = squad_dataset['train']
valid_dataset = squad_dataset['validation']

print('# of train data : {}'.format(len(train_dataset)))
print('# of vaild data : {}'.format(len(valid_dataset)))

batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_length = 255
min_freq = 2
use_bos = True
use_eos = True
print("device : ", device)

loader = SQuAD1Dataloader(train_dataset, valid_dataset, batch_size=batch_size, 
                          device=device, max_length=max_length, min_freq=min_freq,
                          use_bos=use_bos, use_eos=use_eos)
print('\nFinish making the dataloader')
print("batch_size : ", batch_size)
print("max_length : ", max_length)
print('number of used train data ~ {}'.format((len(loader.train_iter)) * batch_size))
print('number of used vaild data ~ {}'.format((len(loader.valid_iter)) * batch_size))

vocab = loader.text.vocab
vocab_list = list(vocab.stoi.keys())
print('number of vocab : {}'.format(len(vocab_list)))

# number of train data : 87599
# number of vaild data : 10570
# device :  cuda

# Finish making the dataloader
# batch_size :  128
# max_length :  255
# number of used train data ~ 81536
# number of used vaild data ~ 9856
# number of vocab : 86580

# of train data : 87599
# of vaild data : 10570
device :  cuda

Finish making the dataloader
batch_size :  128
max_length :  255
number of used train data ~ 81536
number of used vaild data ~ 9856
number of vocab : 86580



Before resolving these differences, you will need to define your evaluation function to correctly evaluate how well your model is doing. Note that the evaluation was very straightforward in Assignment 1's sentiment classification (it is either positive or negative) while it is a bit complicated in SQuAD. We will use the evaluation function provided by `datasets`. You can access to it via the following code.  

In [ ]:
from datasets import load_metric
squad_metric = load_metric('squad')

You can also easily learn about how to use the function by simply typing the function:

In [8]:
squad_metric

Metric(name: "squad", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD scores (F1 and EM).
Args:
    predictions: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair as given in the references (see below)
        - 'prediction_text': the text of the answer
    references: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair (see above),
        - 'answers': a Dict in the SQuAD dataset format
            {
                'text': list of possible texts for the answer, as a list of strings
                'answer_start': list of start positions for the answer, as a list of ints
   

**Problem 3.1** *(10 points)* Let's resolve the first issue here. Hence, for now, assume that your only input is context and you want to obtain the answer without seeing the question. While this may seem to be a non-sense, actually it can be considered as modeling the prior $\text{Prob}(a|c)$ before observing $q$ (we ultimately want $\text{Prob}(a|q,c)$). Transform your model into a token classification model by imposing $\text{softmax}$ over the tokens instead of predefined classes. You will need to do this twice for each of start and end. Report the accuracy (using the metric above) on `squad_dataset['validation']`. 

**Answer 3.1**



In [ ]:
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class ClassificationLSTMModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, n_layers, n_label, emb_dropout, rnn_dropout, bidirectional, enable_layer_norm, device):
        super(ClassificationLSTMModel, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, 
                            hidden_size=hidden_dim, 
                            num_layers=n_layers, 
                            dropout=rnn_dropout, 
                            bidirectional=bidirectional)
      
        n_direction = 2 if bidirectional else 1
        self.fc_start = nn.Linear(hidden_dim*n_direction, n_label, bias=True)
        self.fc_end = nn.Linear(hidden_dim*n_direction, n_label, bias=True)

        # Layer_normalization
        if enable_layer_norm:
            self.enable_layer_norm = enable_layer_norm
            self.emb_layer_norm = nn.LayerNorm(embedding_dim)

        self.emb_dropout = nn.Dropout(emb_dropout)
        self.fc_dropout = nn.Dropout(rnn_dropout)
        self.bidirectional = bidirectional
        self.device = device

    def forward(self, input_tensor, src_seq_lens):
        emb = self.embedding(input_tensor) # emb.shape = batch * len * hidden

        # Layer_normalization
        if self.enable_layer_norm:
            emb = self.emb_layer_norm(emb)

        emb = self.emb_dropout(emb)
        emb = emb.transpose(0, 1) # emb.shape = len * batch * hidden

        # n_direction = 2 if bidirectional else 1
        # hidden = torch.zeros(n_layers*n_direction, context.shape[0], hidden_dim, requires_grad=True).to(self.device)
        # cell = torch.zeros(n_layers*n_direction, context.shape[0], hidden_dim, requires_grad=True).to(self.device)

        # nn.LSTM
        packed = pack_padded_sequence(emb, src_seq_lens.tolist(), batch_first=False)
        outs, (hidden, cell) = self.lstm(packed)
        outs, out_lens = pad_packed_sequence(outs, batch_first=False)

        if self.bidirectional:
            hidden = torch.stack([hidden[-2], hidden[-1]], dim=0)
        else:
            hidden = hidden[-1].unsqueeze(dim=0)
        hidden = hidden.transpose(0, 1)
        hidden = hidden.contiguous().view(hidden.shape[0], -1)

        hidden = self.fc_dropout(hidden)
        logits_start = self.fc_start(hidden)
        logits_end = self.fc_end(hidden)
        return (logits_start, logits_end)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device : ", device)

# Vocabulary : Use vocab_list

# Construct the LSTM Model
embedding_dim = 128 # usually bigger, e.g. 128
hidden_dim = 256
n_layers = 2
n_label = max_length
emb_dropout = 0.5
rnn_dropout = 0.5
bidirectional = True
enable_layer_norm = True
rnnmodel = ClassificationLSTMModel(embedding_dim, hidden_dim, n_layers, n_label, emb_dropout, rnn_dropout, bidirectional, enable_layer_norm, device).to(device)

print("batch_size : ", batch_size)
print("max_length : ", max_length)
print("embedding_dim : ", embedding_dim)
print("hidden_dim : ", hidden_dim)
print("n_layers : ", n_layers)
print("emb_dropout : ", emb_dropout)
print("rnn_dropout_and_fc_dropout : ", rnn_dropout)
if bidirectional:
    print("bidirectional : True")
else:
    print("bidirectional : False")
if enable_layer_norm:
    print("enable_layer_norm : True")
else:
    print("enable_layer_norm : False")

# Construct the data loader
train_iter = loader.train_iter
valid_iter = loader.valid_iter

train_id_list = loader.train_id_list
valid_id_list = loader.valid_id_list
train_reference = loader.train_reference
valid_reference = loader.valid_reference

print(len(train_id_list), len(valid_id_list), len(train_reference), len(valid_reference))

# Training
learning_rate = 1e-3
print("learning_rate : ", learning_rate)

PAD_IDX = vocab.stoi['<pad>']
cel = nn.CrossEntropyLoss(ignore_index=PAD_IDX) # Ignore Padding
# optimizer = torch.optim.SGD(rnnmodel.parameters(), lr=1e-1)
optimizer = torch.optim.Adam(rnnmodel.parameters(), lr=learning_rate)

epochs = 30
max_norm = 5

# Evaluate
squad_metric = load_metric('squad')


for epoch in tqdm(range(epochs)):
    train_loss = 0
    train_accuracy = 0.0
    train_data_num = 0
    train_prediction = list()
    for train_i, train_batch in enumerate(train_iter):
        context, context_length = train_batch.context
        question, question_length = train_batch.question # Unused
        answer_start = train_batch.answer_start
        answer_end = train_batch.answer_end
        train_id_index = train_batch.id_index

        logits_start, logits_end = rnnmodel(context, context_length)

        optimizer.zero_grad() # reset process
        loss = cel(logits_start, answer_start) + cel(logits_end, answer_end) # Loss, a.k.a L
        loss.backward() # compute gradients
        # print(torch.norm(rnnmodel.lstm.weight_hh_l0.grad), loss.item())
        torch.nn.utils.clip_grad_norm_(rnnmodel.parameters(), max_norm) # gradent clipping
        optimizer.step() # update parameters
        train_loss += loss.item()
        
        _, train_start_preds = torch.max(logits_start, 1)
        _, train_end_preds = torch.max(logits_end, 1)
        train_accuracy += ((train_start_preds == answer_start) * (train_end_preds == answer_end)).sum().float()

        train_data_num += context.shape[0]

        # for train_j in range(context.shape[0]):
        #     pred_text = ""
        #     start = train_start_preds[train_j]
        #     end = train_end_preds[train_j]
        #     if start < end:
        #         pred_text = [vocab_list[text_id] for text_id in context[train_j][start:end+1]]
        #         pred_text = " ".join(pred_text)
        #     train_prediction.append({'id':train_id_list[train_id_index[train_j]], 'prediction_text':pred_text})

    # train_result = squad_metric.compute(predictions=train_prediction, references=train_reference)
    print('train:: Epoch:', '%04d' % (epoch + 1), 
          'cost =', '{:.6f},'.format(train_loss / train_data_num), 
          'argmax acc =', '{:.6f}'.format(train_accuracy / train_data_num),
        #   'other_squad_metric : ', train_result
          )
        
    if (epoch + 1) % 1 == 0:
        with torch.no_grad():
            valid_loss = 0
            valid_accuracy = 0.0
            valid_data_num = 0
            valid_prediction = list()
            for valid_i, valid_batch in enumerate(valid_iter):
                context, context_length = valid_batch.context
                question, question_length = valid_batch.question # Unused
                answer_start = valid_batch.answer_start
                answer_end = valid_batch.answer_end
                valid_id_index = valid_batch.id_index

                logits_start, logits_end = rnnmodel(context, context_length)

                loss = cel(logits_start, answer_start) + cel(logits_end, answer_end) # Loss, a.k.a L
                valid_loss += loss.item()

                _, valid_start_preds = torch.max(logits_start, 1)
                _, valid_end_preds = torch.max(logits_end, 1)
                valid_accuracy += ((valid_start_preds == answer_start) * (valid_end_preds == answer_end)).sum().float()

                valid_data_num += context.shape[0]

                for valid_j in range(context.shape[0]):
                    pred_text = ""
                    start = valid_start_preds[valid_j]
                    end = valid_end_preds[valid_j]
                    if start < end:
                        pred_text = [vocab_list[text_id] for text_id in context[valid_j][start:end+1]]
                        pred_text = " ".join(pred_text)
                    valid_prediction.append({'id':valid_id_list[valid_id_index[valid_j]], 'prediction_text':pred_text})
                
            valid_result = squad_metric.compute(predictions=valid_prediction, references=valid_reference)
            print('valid:: Epoch:', '%04d' % (epoch + 1), 
                  'cost =', '{:.6f},'.format(valid_loss / valid_data_num), 
                  'argmax acc =', '{:.6f},'.format(valid_accuracy / valid_data_num),
                  'other_squad_metric : ', valid_result)
            


device :  cuda
batch_size :  128
max_length :  255
embedding_dim :  128
hidden_dim :  256
n_layers :  2
emb_dropout :  0.5
rnn_dropout_and_fc_dropout :  0.5
bidirectional : True
enable_layer_norm : True
81507 9853 81507 9853
learning_rate :  0.001


In [ ]:
# Should match both the start index and the end index
# Random :: 1/256 * 1/256 = 0.0000152587890625 Accuracy

# device :  cuda
# # of vocab : 86580
# batch_size :  128
# max_length :  255
# embedding_dim :  128
# hidden_dim :  256
# n_layers :  2
# bidirectional : True
# learning_rate :  0.001

# train:: Epoch: 0001 cost = 0.079758, acc = 0.004233
# valid:: Epoch: 0001 cost = 0.079524, acc = 0.007123
# train:: Epoch: 0002 cost = 0.079234, acc = 0.010294
# valid:: Epoch: 0002 cost = 0.079435, acc = 0.010786
# train:: Epoch: 0003 cost = 0.079000, acc = 0.014048
# valid:: Epoch: 0003 cost = 0.079326, acc = 0.014652
# train:: Epoch: 0004 cost = 0.078740, acc = 0.016686
# valid:: Epoch: 0004 cost = 0.079321, acc = 0.015263
# train:: Epoch: 0005 cost = 0.078444, acc = 0.018661
# valid:: Epoch: 0005 cost = 0.079526, acc = 0.015975
# train:: Epoch: 0006 cost = 0.078154, acc = 0.020550
# valid:: Epoch: 0006 cost = 0.079506, acc = 0.016992
# train:: Epoch: 0007 cost = 0.077840, acc = 0.021728
# valid:: Epoch: 0007 cost = 0.079699, acc = 0.018519
# train:: Epoch: 0008 cost = 0.077550, acc = 0.023434
# valid:: Epoch: 0008 cost = 0.079824, acc = 0.019129
# train:: Epoch: 0009 cost = 0.077251, acc = 0.024611
# valid:: Epoch: 0009 cost = 0.079951, acc = 0.018315
# train:: Epoch: 0010 cost = 0.076993, acc = 0.024722
# valid:: Epoch: 0010 cost = 0.080069, acc = 0.019333 -> max valid acc ~1/50
# train:: Epoch: 0011 cost = 0.076738, acc = 0.026550
# valid:: Epoch: 0011 cost = 0.080116, acc = 0.018620

**Problem 3.2** *(10 points)*  Now let's resolve the second issue, by simply concatenating the two inputs into one sequence. The simplest way would be to append the the question at the start *OR* the end of the context. If you put it at the start, you will need to shift the start and the end positions of the answer accordingly. If you put it at the end, it will be necesary to use bidirectional LSTM for the context to be aware of what is ahead (though it is recommended to use bidirectional LSTM even if the question is appended at the start). Whichever you choose, carry it out and report the accuracy. How does it differ from 3.1?

In [ ]:
# Put the question sentence before the context sentence (for Prob 3.2+)
# Use <CLS> token to separate two sentences

import torchtext
from torchtext.legacy import data
from torchtext.legacy import datasets
from torchtext.legacy.data import BucketIterator


class SQuAD2Dataset(data.Dataset):
  """
  Defines a dataset for squad1.0.
  """
  
  @staticmethod
  def sort_key(ex):
    return data.interleave_keys(len(ex.context_question))

  def __init__(self, data_list, fields, use_bos=True, max_length=None, **kwargs):
    if not isinstance(fields[0], (tuple, list)):
      fields = [
                # ('context', fields[0]), 
                # ('question', fields[1]), 
                ('context_question', fields[0]), # For Problem 3.2+, put the question after the context
                ('answer_start', fields[1]), 
                ('answer_end', fields[2]), 
                ('id_index', fields[3])
                ]

    examples = []
    nonalpha_list = find_nonalpha_list(data_list)

    self.id_list = list()
    self.reference = list()

    for _, example in enumerate(data_list):
        out = preprocess(example, nonalpha_list)
        # use data if the answer exists
        if max_length and max_length < max(len(out['context']), len(out['question'])):
            continue
        if 'answers' in out.keys():
            answer_start = out['answers'][0]['start'] # Use index 0 for instant valid accuracy
            answer_end = out['answers'][0]['end'] # Use index 0 for instant valid accuracy
            if use_bos:
                answer_start += 1 # for <BOS> token
                answer_end += 1 # for <BOS> token
            examples.append(data.Example.fromlist([
                                                #    out['context'], 
                                                #    out['question'], 
                                                   out['context'] + ["<CLS>"] + out['question'], # Use <CLS> token
                                                   answer_start,
                                                   answer_end,
                                                   len(self.id_list)], 
                                                  fields))
            self.id_list.append(out['id'])
            self.reference.append({'id':example['id'], 'answers':example['answers']})

    super(SQuAD2Dataset, self).__init__(examples, fields, **kwargs)


class SQuAD2Dataloader():
  """
  Make the dataloader for SQuAD 1.0
  """
  def __init__(self, train_data=None, valid_data=None, batch_size=64, device='cpu', 
                max_length=255, min_freq=2, fix_length=None,
                use_bos=True, use_eos=True, shuffle=True
              ):

    super(SQuAD2Dataloader, self).__init__()

    self.text = data.Field(sequential=True, use_vocab=True, batch_first=True, 
                           include_lengths=True, fix_length=fix_length, 
                           init_token='<BOS>' if use_bos else None, 
                           eos_token='<EOS>' if use_eos else None
                          )
    self.answer_start = data.Field(sequential = False, use_vocab = False)
    self.answer_end = data.Field(sequential = False, use_vocab = False)
    self.id_index = data.Field(sequential = False, use_vocab = False)
    
    train = SQuAD2Dataset(data_list=train_data, 
                          fields = [
                                    # ('context', self.text),
                                    # ('question', self.text),
                                    ('context_question', self.text),
                                    ('answer_start', self.answer_start),
                                    ('answer_end', self.answer_end),
                                    ('id_index', self.id_index)
                                    ], 
                          max_length = max_length
                          )
    valid = SQuAD2Dataset(data_list=valid_data, 
                          fields = [
                                    # ('context', self.text),
                                    # ('question', self.text),
                                    ('context_question', self.text),
                                    ('answer_start', self.answer_start),
                                    ('answer_end', self.answer_end),
                                    ('id_index', self.id_index)
                                    ], 
                          max_length = max_length
                          )
    self.train_id_list = train.id_list
    self.valid_id_list = valid.id_list

    self.train_reference = train.reference
    self.valid_reference = valid.reference
    
    self.train_iter = data.BucketIterator(train, batch_size=batch_size,
                                          device=device,
                                          shuffle=shuffle,
                                          sort_key=lambda x: len(x.context_question), 
                                          sort_within_batch = True
                                          )
    self.valid_iter = data.BucketIterator(valid, batch_size=batch_size,
                                          device=device,
                                          shuffle=shuffle,
                                          sort_key=lambda x: len(x.context_question),
                                          sort_within_batch = True
                                          )
    
    self.text.build_vocab(train)


train_dataset = squad_dataset['train']
valid_dataset = squad_dataset['validation']

print('# of train data : {}'.format(len(train_dataset)))
print('# of vaild data : {}'.format(len(valid_dataset)))

batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_length = 255
min_freq = 2
use_bos = True
use_eos = True
print("device : ", device)

loader = SQuAD2Dataloader(train_dataset, valid_dataset, batch_size=batch_size, 
                          device=device, max_length=max_length, min_freq=min_freq,
                          use_bos=use_bos, use_eos=use_eos)
print('\nFinish making the dataloader')
print("batch_size : ", batch_size)
print("max_length : ", max_length)
print('# of used train data ~ {}'.format((len(loader.train_iter)) * batch_size))
print('# of used vaild data ~ {}'.format((len(loader.valid_iter)) * batch_size))

vocab = loader.text.vocab
vocab_list = list(vocab.stoi.keys())
print('# of vocab : {}'.format(len(vocab_list)))

# # of train data : 87599
# # of vaild data : 10570
# device :  cuda

# Finish making the dataloader
# batch_size :  128
# max_length :  255
# # of used train data ~ 81536
# # of used vaild data ~ 9856
# # of vocab : 86580

**Answer 3.2**



In [ ]:
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device : ", device)

# Vocabulary : Use vocab_list

# Construct the LSTM Model
embedding_dim = 128 # usually bigger, e.g. 128
hidden_dim = 256
n_layers = 2
n_label = max_length
emb_dropout = 0.5
rnn_dropout = 0.5
bidirectional = True
enable_layer_norm = True
rnnmodel = ClassificationLSTMModel(embedding_dim, hidden_dim, n_layers, n_label, emb_dropout, rnn_dropout, bidirectional, enable_layer_norm, device).to(device)

print("batch_size : ", batch_size)
print("max_length : ", max_length)
print("embedding_dim : ", embedding_dim)
print("hidden_dim : ", hidden_dim)
print("n_layers : ", n_layers)
print("emb_dropout : ", emb_dropout)
print("rnn_dropout_and_fc_dropout : ", rnn_dropout)
if bidirectional:
    print("bidirectional : True")
else:
    print("bidirectional : False")
if enable_layer_norm:
    print("enable_layer_norm : True")
else:
    print("enable_layer_norm : False")

# Construct the data loader
train_iter = loader.train_iter
valid_iter = loader.valid_iter

train_id_list = loader.train_id_list
valid_id_list = loader.valid_id_list
train_reference = loader.train_reference
valid_reference = loader.valid_reference

print(len(train_id_list), len(valid_id_list), len(train_reference), len(valid_reference))

# Training
learning_rate = 1e-3
print("learning_rate : ", learning_rate)

PAD_IDX = vocab.stoi['<pad>']
cel = nn.CrossEntropyLoss(ignore_index=PAD_IDX) # Ignore Padding
# optimizer = torch.optim.SGD(rnnmodel.parameters(), lr=1e-1)
optimizer = torch.optim.Adam(rnnmodel.parameters(), lr=learning_rate)

epochs = 30
max_norm = 5

# Evaluate
squad_metric = load_metric('squad')


for epoch in tqdm(range(epochs)):
    train_loss = 0
    train_accuracy = 0.0
    train_data_num = 0
    train_prediction = list()
    for train_i, train_batch in enumerate(train_iter):
        context, context_length = train_batch.context_question
        answer_start = train_batch.answer_start
        answer_end = train_batch.answer_end
        train_id_index = train_batch.id_index

        logits_start, logits_end = rnnmodel(context, context_length)

        optimizer.zero_grad() # reset process
        loss = cel(logits_start, answer_start) + cel(logits_end, answer_end) # Loss, a.k.a L
        loss.backward() # compute gradients
        # print(torch.norm(rnnmodel.lstm.weight_hh_l0.grad), loss.item())
        torch.nn.utils.clip_grad_norm_(rnnmodel.parameters(), max_norm) # gradent clipping
        optimizer.step() # update parameters
        train_loss += loss.item()
        
        _, train_start_preds = torch.max(logits_start, 1)
        _, train_end_preds = torch.max(logits_end, 1)
        train_accuracy += ((train_start_preds == answer_start) * (train_end_preds == answer_end)).sum().float()

        train_data_num += context.shape[0]

        for train_j in range(context.shape[0]):
            pred_text = ""
            start = train_start_preds[train_j]
            end = train_end_preds[train_j]
            if start < end:
                pred_text = [vocab_list[text_id] for text_id in context[train_j][start:end+1]]
                pred_text = " ".join(pred_text)
            train_prediction.append({'id':train_id_list[train_id_index[train_j]], 'prediction_text':pred_text})

    train_result = squad_metric.compute(predictions=train_prediction, references=train_reference)
    print('train:: Epoch:', '%04d' % (epoch + 1), 
          'cost =', '{:.6f},'.format(train_loss / train_data_num), 
        #   'argmax acc =', '{:.6f}'.format(train_accuracy / train_data_num),
          'other_squad_metric : ', train_result)
        
    if (epoch + 1) % 1 == 0:
        with torch.no_grad():
            valid_loss = 0
            valid_accuracy = 0.0
            valid_data_num = 0
            valid_prediction = list()
            for valid_i, valid_batch in enumerate(valid_iter):
                context, context_length = valid_batch.context_question
                question, question_length = valid_batch.question # Unused
                answer_start = valid_batch.answer_start
                answer_end = valid_batch.answer_end
                valid_id_index = valid_batch.id_index

                logits_start, logits_end = rnnmodel(context, context_length)

                loss = cel(logits_start, answer_start) + cel(logits_end, answer_end) # Loss, a.k.a L
                valid_loss += loss.item()

                _, valid_start_preds = torch.max(logits_start, 1)
                _, valid_end_preds = torch.max(logits_end, 1)
                valid_accuracy += ((valid_start_preds == answer_start) * (valid_end_preds == answer_end)).sum().float()

                valid_data_num += context.shape[0]

                for valid_j in range(context.shape[0]):
                    pred_text = ""
                    start = valid_start_preds[valid_j]
                    end = valid_end_preds[valid_j]
                    if start < end:
                        pred_text = [vocab_list[text_id] for text_id in context[valid_j][start:end+1]]
                        pred_text = " ".join(pred_text)
                    valid_prediction.append({'id':valid_id_list[valid_id_index[valid_j]], 'prediction_text':pred_text})
                
            valid_result = squad_metric.compute(predictions=valid_prediction, references=valid_reference)
            print('valid:: Epoch:', '%04d' % (epoch + 1), 
                  'cost =', '{:.6f},'.format(valid_loss / valid_data_num), 
                #   'argmax acc =', '{:.6f},'.format(valid_accuracy / valid_data_num),
                  'other_squad_metric : ', valid_result)
            


In [ ]:
device :  cuda
# of vocab : 86580
batch_size :  128
max_length :  255
embedding_dim :  256
hidden_dim :  512
n_layers :  2
bidirectional : True
learning_rate :  0.001
13%
13/100 [33:49<3:46:29, 156.21s/it]
train:: Epoch: 0001 cost = 0.079619, acc = 0.006944
valid:: Epoch: 0001 cost = 0.079696, acc = 0.012922
train:: Epoch: 0002 cost = 0.079165, acc = 0.013582
valid:: Epoch: 0002 cost = 0.079346, acc = 0.015466
train:: Epoch: 0003 cost = 0.078792, acc = 0.017164
valid:: Epoch: 0003 cost = 0.079322, acc = 0.016280
train:: Epoch: 0004 cost = 0.078353, acc = 0.020182
valid:: Epoch: 0004 cost = 0.079481, acc = 0.017908
train:: Epoch: 0005 cost = 0.077817, acc = 0.022685
valid:: Epoch: 0005 cost = 0.079363, acc = 0.019638
train:: Epoch: 0006 cost = 0.077266, acc = 0.025286
valid:: Epoch: 0006 cost = 0.079417, acc = 0.019027
train:: Epoch: 0007 cost = 0.076714, acc = 0.026452
valid:: Epoch: 0007 cost = 0.079670, acc = 0.018722
train:: Epoch: 0008 cost = 0.076170, acc = 0.026881
valid:: Epoch: 0008 cost = 0.080020, acc = 0.018112
train:: Epoch: 0009 cost = 0.075587, acc = 0.029507
valid:: Epoch: 0009 cost = 0.080270, acc = 0.020350
train:: Epoch: 0010 cost = 0.075055, acc = 0.029605
valid:: Epoch: 0010 cost = 0.080645, acc = 0.020045
train:: Epoch: 0011 cost = 0.074553, acc = 0.031519
valid:: Epoch: 0011 cost = 0.080962, acc = 0.017806
train:: Epoch: 0012 cost = 0.074054, acc = 0.032329
valid:: Epoch: 0012 cost = 0.081124, acc = 0.019129
train:: Epoch: 0013 cost = 0.073636, acc = 0.032574
valid:: Epoch: 0013 cost = 0.081502, acc = 0.018315

In [ ]:
device :  cuda
# of vocab : 86580
batch_size :  128
max_length :  255
embedding_dim :  128
hidden_dim :  256
n_layers :  2
bidirectional : True
learning_rate :  0.001
23%
23/100 [20:52<1:09:54, 54.47s/it]
train:: Epoch: 0001 cost = 0.079600, acc = 0.007619
valid:: Epoch: 0001 cost = 0.079375, acc = 0.015059
train:: Epoch: 0002 cost = 0.078746, acc = 0.017029
valid:: Epoch: 0002 cost = 0.079089, acc = 0.017094
train:: Epoch: 0003 cost = 0.077435, acc = 0.024035
valid:: Epoch: 0003 cost = 0.079096, acc = 0.020452
train:: Epoch: 0004 cost = 0.074976, acc = 0.032267
valid:: Epoch: 0004 cost = 0.080308, acc = 0.020045
train:: Epoch: 0005 cost = 0.070933, acc = 0.041861
valid:: Epoch: 0005 cost = 0.082924, acc = 0.015263
train:: Epoch: 0006 cost = 0.064731, acc = 0.058449
valid:: Epoch: 0006 cost = 0.087871, acc = 0.012210
train:: Epoch: 0007 cost = 0.056475, acc = 0.090802
valid:: Epoch: 0007 cost = 0.094999, acc = 0.008140
train:: Epoch: 0008 cost = 0.047037, acc = 0.147779
valid:: Epoch: 0008 cost = 0.104818, acc = 0.007326
train:: Epoch: 0009 cost = 0.037530, acc = 0.239709
valid:: Epoch: 0009 cost = 0.117320, acc = 0.005698
train:: Epoch: 0010 cost = 0.028805, acc = 0.358301
valid:: Epoch: 0010 cost = 0.130119, acc = 0.005393
train:: Epoch: 0011 cost = 0.021250, acc = 0.492866
valid:: Epoch: 0011 cost = 0.140989, acc = 0.004375
train:: Epoch: 0012 cost = 0.015138, acc = 0.627100
valid:: Epoch: 0012 cost = 0.153281, acc = 0.005698
train:: Epoch: 0013 cost = 0.010398, acc = 0.749727
valid:: Epoch: 0013 cost = 0.163834, acc = 0.003867
train:: Epoch: 0014 cost = 0.007055, acc = 0.842112
valid:: Epoch: 0014 cost = 0.173189, acc = 0.004375
train:: Epoch: 0015 cost = 0.004955, acc = 0.898868
valid:: Epoch: 0015 cost = 0.181411, acc = 0.003663
train:: Epoch: 0016 cost = 0.003635, acc = 0.932202
valid:: Epoch: 0016 cost = 0.189040, acc = 0.003968
train:: Epoch: 0017 cost = 0.003136, acc = 0.939085
valid:: Epoch: 0017 cost = 0.194765, acc = 0.004172
train:: Epoch: 0018 cost = 0.003493, acc = 0.920534
valid:: Epoch: 0018 cost = 0.198848, acc = 0.003765
train:: Epoch: 0019 cost = 0.003538, acc = 0.912314
valid:: Epoch: 0019 cost = 0.204216, acc = 0.003663
train:: Epoch: 0020 cost = 0.002579, acc = 0.941907
valid:: Epoch: 0020 cost = 0.209405, acc = 0.003460
train:: Epoch: 0021 cost = 0.001781, acc = 0.964163
valid:: Epoch: 0021 cost = 0.214543, acc = 0.004274
train:: Epoch: 0022 cost = 0.001338, acc = 0.976125
valid:: Epoch: 0022 cost = 0.218680, acc = 0.004172
train:: Epoch: 0023 cost = 0.001490, acc = 0.968776
valid:: Epoch: 0023 cost = 0.221427, acc = 0.004375

In [ ]:
device :  cuda
# of vocab : 86580
batch_size :  128
max_length :  255
embedding_dim :  128
hidden_dim :  256
n_layers :  2
bidirectional : True
learning_rate :  0.001
11%
11/100 [13:27<1:48:54, 73.42s/it]
train:: Epoch: 0001 cost = 0.079707, acc = 0.005570
valid:: Epoch: 0001 cost = 0.079532, acc = 0.007835
train:: Epoch: 0002 cost = 0.079195, acc = 0.010968
valid:: Epoch: 0002 cost = 0.079389, acc = 0.014042
train:: Epoch: 0003 cost = 0.078919, acc = 0.014490
valid:: Epoch: 0003 cost = 0.079269, acc = 0.014449
train:: Epoch: 0004 cost = 0.078588, acc = 0.016992
valid:: Epoch: 0004 cost = 0.079284, acc = 0.017705
train:: Epoch: 0005 cost = 0.078201, acc = 0.019238
valid:: Epoch: 0005 cost = 0.079297, acc = 0.016585
train:: Epoch: 0006 cost = 0.077760, acc = 0.020992
valid:: Epoch: 0006 cost = 0.079323, acc = 0.017806
train:: Epoch: 0007 cost = 0.077337, acc = 0.022182
valid:: Epoch: 0007 cost = 0.079581, acc = 0.016077
train:: Epoch: 0008 cost = 0.076933, acc = 0.023213
valid:: Epoch: 0008 cost = 0.079590, acc = 0.019333
train:: Epoch: 0009 cost = 0.076559, acc = 0.024857
valid:: Epoch: 0009 cost = 0.079766, acc = 0.018824
train:: Epoch: 0010 cost = 0.076169, acc = 0.025556
valid:: Epoch: 0010 cost = 0.079945, acc = 0.018315
train:: Epoch: 0011 cost = 0.075802, acc = 0.026734
valid:: Epoch: 0011 cost = 0.080164, acc = 0.017806

## 4. LSTM + Attention for SQuAD

**Problem 4.1** *(20 points)* Here, we will be appending an attention layer on top of LSTM outputs. We will use a single-head attention sublayer from Transformer. That is, you will implement 
$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d}}\right)V,$$
where $Q, K, V$ is obtained by the linear transformation of the hidden states of the LSTM outputs $H$, i.e. $Q = HW^Q, K=HW^K, V=HW^V$ ($W^Q, W^K, W^V \in \mathbb{R}^{d \times d}$ are trainable weights). Note that the output of $\text{Attention}$ layer has the same dimension as $H$, so you can directly append your token classification layer on top of it. Report the accuracy and compare it with 3.2.



**Answer 4.1**



**Problem 4.2** *(10 points)* On top of the attention layer, let's add another layer of (bi-directional) LSTM. So this will look like a *sandwich* where the LSTM is bread and the attention is ham. How does it affect the accuracy? Explain why do you think this happens. 

**Answer 4.2**



## 5. Attention is All You Need

**Problem 5.1 (bonus)** *(20 points)*  Implement full Transformer encoder to entirely replace LSTMs. You are allowed to copy and paste code from [*Annotated Transformer*](https://nlp.seas.harvard.edu/2018/04/03/attention.html) (but nowhere else). Report the accuracy and explain what seems to happening with attetion-only model compared to LSTM+Attention model(s). 


**Answer 5.1**


**Problem 5.2 (bonus)** *(10 points)* Replace Transformer's sinusoidal position encoding with a fixed-length (of 256) position embedding. That is, you will create a 256-by-$d$ trainable parameter matrix for the position encoding that replaces the variable-length sinusoidal encoding. What is the clear disdvantage of this approach? Report the accuracy and compare it with 5.1. Note that this also has a clear advantage, as we will see in our future lecture on Pretrained Language Model, and more specifically, BERT (Devlin et al., 2018).

**Answer 5.2**

